# Web-Scraping Process

First part for the project was to obtain the music catalogue reference. 

In [ ]:
#Libraries

import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

import regex as re

import time

import matplotlib.pyplot as plt

import itertools

# Get Genre Tags, Number of tracks

url_pattern = 'https://nodata.tv/blog/page/%s'

# List of blog pages

author_test=[]
album_test=[]
year_test=[]
link_test=[]
for item in range(1, 10):
    time.sleep(2)
    url = url_pattern%item
    html = requests.get(url)
    html = html.content
    soup = BeautifulSoup(html, 'lxml')
    # Post Title
    post_title = soup.find_all('a', {'class':'title'})
    post_title_clean = [title.text.strip() for title in post_title]
    # Post link
    link = [post_title[i]['href'] for i in range(len(post_title))]
    #print(link)
    link_test.append(link)
    # Author
    author = [re.findall('.+(?=\/)', item)[0] for item in post_title_clean]
    author_test.append(author)
    #Album
    album = [re.findall('(?<=\/).+(?=\[)', item)[0] for item in post_title_clean]
    album_test.append(album)
    # Year
    year = [re.findall('(?<=\[)\d+', item) for item in post_title_clean]
    year_test.append(year)
    # Genre list
    #genre = soup.find_all('a', {'rel':'category tag'})
    #genre_clean = [genre.text.strip() for genre in genre]
    # Number of tracks
    #tracks = soup.find_all("ol")
    #tracks_clean = [track.text.strip().split("\n") for track in tracks]
    #print(tracks_clean)
    #track_number = len(tracks_clean)
    #genre_test.append(genre_clean)
    #track_test.append(track_number)


# Taking lists out of lists by page    
# Reference: https://stackoverflow.com/questions/716477/join-list-of-lists-in-python

author = (list(itertools.chain.from_iterable(author_test)))
album = (list(itertools.chain.from_iterable(album_test)))
year = (list(itertools.chain.from_iterable(year_test)))
link = (list(itertools.chain.from_iterable(link_test)))


genre_test = []
track_test = []
for item in link:
    time.sleep(2)
    url = item
    html = requests.get(url)
    html = html.content
    soup = BeautifulSoup(html, 'lxml')
    # Genre list
    genre = soup.find_all('a', {'rel':'category tag'})
    genre_clean = [genre.text.strip() for genre in genre]
    genre_test.append(genre_clean)
    # Number of tracks
    tracks = soup.find_all("ol")
    tracks_clean = [track.text.strip().split("\n") for track in tracks]
    #print(tracks_clean)
    track_number = len(tracks_clean[0])
    track_test.append(track_number)

# Post_Id
post_id = [re.findall('(?<=.tv/)\d+', item)[0] for item in link]

df_test = pd.DataFrame(list(zip(post_id, author, album, year, genre_test, track_test, link)),
                  columns = ['Post_ID', 'Author', 'Album', 'Year', 'Genre', 'No. Tracks', 'Link'], index = None)
    


df_test["Year"] = df_test["Year"].explode()

print(df_test.info())

In [ ]:
# df_test.to_csv('df_basic_info.txt', index=False)

In [ ]:
df_genres = df_test[['Post_ID', 'Genre']]

In [ ]:
genre_dict = {}
for i in df_genres['Genre']:
    for j in i:
        if j not in genre_dict:
            genre_dict[j]=1
        else:
            genre_dict[j]+=1

In [ ]:
genre_freq = pd.DataFrame.from_dict(genre_dict, orient='index', columns=['Frequency'])
genre_freq = genre_freq.reset_index()
genre_freq = genre_freq.rename(columns={'index':'Genre'})
# genre_freq

In [ ]:
# Reference : https://www.python-graph-gallery.com/circular-barplot-basic

import numpy as np

# fig size
plt.figure(figsize=(30,10))

# plot polar axis
ax = plt.subplot(111, polar=True)

# remove grid
plt.axis('off')

# coordinate limits 
upperLimit = 100
lowerLimit = 20

# Compute max and min
max_ = genre_freq['Frequency'].max()

# 
#
#

slope = (max_ - lowerLimit) / max_
heights = slope * genre_freq.Frequency + lowerLimit

width = 2*np.pi / len(genre_freq.index)

indexes = list(range(1, len(genre_freq.index)+1))

angles = [element * width for element in indexes]
angles

# Draw bars
bars = ax.bar(
    x=angles, 
    height=heights, 
    width=width, 
    bottom=lowerLimit,
    linewidth=2, 
    edgecolor="white",
    color="b")

labelPadding = 4

# Add labels
for bar, angle, height, label in zip(bars,angles, heights, genre_freq['Genre']):

    # Labels are rotated. Rotation must be specified in degrees :(
    rotation = np.rad2deg(angle)

    # Flip some labels upside down
    alignment = ""
    if angle >= np.pi/2 and angle < 3*np.pi/2:
        alignment = "right"
        rotation = rotation + 180
    else: 
        alignment = "left"

    # Finally add the labels
    ax.text(
        x=angle, 
        y=lowerLimit + bar.get_height() + labelPadding, 
        s=label, 
        ha=alignment, 
        va='center', 
        rotation=rotation, 
        rotation_mode="anchor",
        fontsize=10)
    
plt.savefig('test01.jpg')    

In [ ]:
 # df_genres.to_csv('df_genres.txt', index=False)

In [ ]:
spotify_artists = df_test['Author']

# Linking scraped data with Spotify's API

The artist were located by name to obtain ID, Followers, Popularity Index

In [ ]:
import requests
import base64
from urllib.parse import urlencode


# Credentials MUST CHANGE BEFORE UPLOADING!
client_id = "<client ID>"
client_secret = "<client secret>"

# Formating credentials for 
dev_cred = f"{client_id}:{client_secret}"

dev_cred_b64 = base64.b64encode(dev_cred.encode())

token_url = "https://accounts.spotify.com/api/token"

method = "POST"

datos_token = {"grant_type":"client_credentials"}

token_header = {"Authorization": f'Basic {dev_cred_b64.decode()}'}

request = requests.post(token_url, data = datos_token, headers = token_header)

datos = request.json()

authorization = {"Authorization":f"Bearer {datos['access_token']}"}

test_df= []

for item in spotify_artists:

    query = urlencode({"q":item,"type":"artist"})

    endpoint = 'https://api.spotify.com/v1/search'

    url_solicitud = f"{endpoint}?{query}"

    busca = requests.get(url_solicitud, headers = authorization)
    
    test_df.append(busca)
    
test_df = [item.json() for item in test_df]

artist_df = pd.json_normalize(test_df)

In [ ]:
machote = [artist_df['artists.items'][i] for i in range(0, len(spotify_artists))]

In [ ]:
machote = (list(itertools.chain.from_iterable(machote)))

In [ ]:
machote = pd.json_normalize(machote)

In [ ]:
machote.info()

In [ ]:
df_test['Author'] = [i.strip() for i in df_test['Author']]

In [ ]:
authors_test = []

for item in df_test['Author']:
    if item in set(machote['name']):
        authors_test.append(item)

In [ ]:
boolean_series = machote['name'].isin(authors_test)

In [ ]:
filtered_machote = machote[boolean_series]

In [ ]:
# df_genres.to_csv('df_genres.txt', index=False)

# filtered_machote['id'].to_csv('id_test.csv', index=False)

In [ ]:
machote_1 = filtered_machote.pivot_table(index=['name'], values=['genres', 'followers.total', 'popularity'])
machote_1 = machote_1.reset_index()

In [ ]:
machote_1

In [ ]:
test_spty = [x for x in machote_1['name'] if x in spotify_artists]

In [ ]:
removers = []
for i in test_spty:
    keeper = machote_1[machote_1['name'] == i].index
    removers.append(keeper)


In [ ]:
machote_1.set_index('name', inplace=True)

In [ ]:
machite_3 = machote_1

In [ ]:
machite_3 = machite_3.reset_index()

In [ ]:
machite_3.to_csv('spotify_info.txt', index=False)

In [ ]:
import seaborn as sns

df_corr = machite_3[['followers.total', 'popularity']].corr()
cmap = sns.diverging_palette(6, 255, as_cmap=True)
sns.heatmap(df_corr, cmap=cmap,annot=True)

In [ ]:
sns.scatterplot(data=machite_3, x='name', y='popularity', size='followers.total', hue='name', legend=False, x_jitter=50, sizes=(100,1000))

plt.xticks(rotation = 90)

fig.size(20,50)

plt.show()